In [1]:
import networkx as nx
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import copy

In [2]:
# Reading back the prepared separate networks
ports = ['1', '2']
years = ['2005', '2010', '2015', '2017']
codes = ['2701', '2709', '2711', '271111', '271121']

G = {(port, year, code): nx.DiGraph() for port in ports for year in years for code in codes}

for gKey in G:
    G[gKey] = nx.read_gpickle("PycharmProjects/EnergyTradeAnalysis/Database/GPickleData/G_" + str(gKey) + ".gp")

In [4]:
# Export Asymmetry Ratios
# DIF(E) = (Exports(A to B) − Imports(B from A))/Exports(A to B)
# ER(E) = (Exports(A to B) − Imports(B from A))/((Exports(A to B) + Imports(B from A))/2)

expAssymRatioDIF = {(year, code): pd.DataFrame() for year in years for code in codes[:3]}
expAssymRatioER = {(year, code): pd.DataFrame() for year in years for code in codes[:3]}

for year in years:
    for code in codes[:3]:
        expAssymRatioDIF[(year, code)] = pd.DataFrame(columns=['Country A', 'Country B', 'Exp: A to B', 'Imp: B from A', 'ExpRatio'])
        expAssymRatioER[(year, code)] = pd.DataFrame(columns=['Country A', 'Country B', 'Exp: A to B', 'Imp: B from A', 'ExpRatio'])

        # for each edge in export network
        for edge in list(G[(ports[1], year, code)].edges(data=True)):
            exportValue = G[(ports[1], year, code)][edge[0]][edge[1]]['weight']
            importValue = 0
            
            # if import network has the same edge
            if G[(ports[0], year, code)].has_edge(edge[0], edge[1]):
                importValue = G[(ports[0], year, code)][edge[0]][edge[1]]['weight']
                    
            if exportValue > 0:
                expRatioDIF = (exportValue - importValue) * 100/ exportValue
                expRatioER = (exportValue - importValue) * 100/ ((exportValue + importValue)/2)
                     
                toAppendDIF = [edge[0], edge[1], exportValue, importValue, expRatioDIF]
                aSeriesDIF = pd.Series(toAppendDIF, index = expAssymRatioDIF[(year, code)].columns)
                expAssymRatioDIF[(year, code)] = expAssymRatioDIF[(year, code)].append(aSeriesDIF, ignore_index=True)
                
                toAppendER = [edge[0], edge[1], exportValue, importValue, expRatioER]
                aSeriesER = pd.Series(toAppendER, index = expAssymRatioER[(year, code)].columns)
                expAssymRatioER[(year, code)] = expAssymRatioER[(year, code)].append(aSeriesER, ignore_index=True)
                

In [6]:
# Import Asymetry Ratios
# DIF(I) = (Imports(B from A) − Exports(A to B))/Imports(B from A)
# ER(I) = (Imports(B from A) − Exports(A to B))/((Imports(B from A) + Exports(A to B))/2)

impAssymRatioDIF = {(year, code): pd.DataFrame() for year in years for code in codes[:3]}
impAssymRatioER = {(year, code): pd.DataFrame() for year in years for code in codes[:3]}

for year in years:
    for code in codes[:3]:
        impAssymRatioDIF[(year, code)] = pd.DataFrame(columns=['Country A', 'Country B', 'Imp: B from A', 'Exp: A to B', 'ImpRatio'])
        impAssymRatioER[(year, code)] = pd.DataFrame(columns=['Country A', 'Country B', 'Imp: B from A', 'Exp: A to B', 'ImpRatio'])
        
        # for each edge in import network
        for edge in list(G[(ports[0], year, code)].edges(data=True)):
            exportValue = 0
            importValue = G[(ports[0], year, code)][edge[0]][edge[1]]['weight']
            
            # if export network has the same edge
            if G[(ports[1], year, code)].has_edge(edge[0], edge[1]):
                exportValue = G[(ports[1], year, code)][edge[0]][edge[1]]['weight']
        
            if importValue > 0:
                impRatioDIF = (importValue - exportValue) * 100 / importValue
                impRatioER = (importValue - exportValue) * 100 / ((exportValue + importValue)/2)
                
                toAppendDIF = [edge[0], edge[1], importValue, exportValue, impRatioDIF]
                aSeriesDIF = pd.Series(toAppendDIF, index = impAssymRatioDIF[(year, code)].columns)
                impAssymRatioDIF[(year, code)] = impAssymRatioDIF[(year, code)].append(aSeriesDIF, ignore_index=True)

                toAppendER = [edge[0], edge[1], importValue, exportValue, impRatioER]
                aSeriesER = pd.Series(toAppendER, index = impAssymRatioER[(year, code)].columns)
                impAssymRatioER[(year, code)] = impAssymRatioER[(year, code)].append(aSeriesER, ignore_index=True)


In [7]:
for difKey in impAssymRatioDIF.keys():
    impAssymRatioDIF[difKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1/DIF_INA' + str(difKey) + '.csv', mode = 'w')
    impAssymRatioER[difKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2/ER_INA' + str(difKey) + '.csv', mode = 'w')

for erKey in expAssymRatioDIF.keys():
    expAssymRatioDIF[erKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1/DIF_ENA' + str(erKey) + '.csv', mode = 'w')
    expAssymRatioER[erKey].to_csv('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2/ER_ENA' + str(erKey) + '.csv', mode = 'w')


In [8]:
# DIF Graph generation for both Export and Import
for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1'):
    with open('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1/' + file, encoding='utf-8', errors='ignore') as csv_file:
        df = pd.read_csv(csv_file, index_col=0)
        if 'ENA' in str(file):
            expRatioList = sorted(df['ExpRatio'].tolist())
            idList = list(df.index.values)
            plt.scatter(idList, expRatioList, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$DIF(E)_{it}^{AB}$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/Real" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()    
        elif 'INA' in str(file):
            impRatioList = sorted(df['ImpRatio'].tolist())
            impLogData = [np.sign(i)*np.log(abs(i)) for i in impRatioList]
            idList = list(df.index.values)
            plt.scatter(idList, impRatioList, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$DIF(I)_{it}^{AB}$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/Real" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()
            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


<Figure size 432x288 with 0 Axes>

In [9]:
# log(DIF) Graph generation for both Export and Import
for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1'):
    with open('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1/' + file, encoding='utf-8', errors='ignore') as csv_file:
        df = pd.read_csv(csv_file, index_col=0)
        if 'ENA' in str(file):
            expRatioList = sorted(df['ExpRatio'].tolist())
            expLogData = [np.sign(i)*np.log(abs(i)) for i in expRatioList]
            idList = list(df.index.values)
            plt.scatter(idList, expLogData, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$log(DIF(E)_{it}^{AB})$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/Log" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()    
        elif 'INA' in str(file):
            impRatioList = sorted(df['ImpRatio'].tolist())
            impLogData = [np.sign(i)*np.log(abs(i)) for i in impRatioList]
            idList = list(df.index.values)
            plt.scatter(idList, impLogData, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$log(DIF(I)_{it}^{AB})$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/Log" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()
            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


<Figure size 432x288 with 0 Axes>

In [10]:
# ER Graph generation for both Export and Import
for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2'):
    with open('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2/' + file, encoding='utf-8', errors='ignore') as csv_file:
        df = pd.read_csv(csv_file, index_col=0)
        if 'ENA' in str(file):
            expRatioList = sorted(df['ExpRatio'].tolist())
            idList = list(df.index.values)
            plt.scatter(idList, expRatioList, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$ER(E)_{it}^{AB}$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/Real" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()    
        elif 'INA' in str(file):
            impRatioList = sorted(df['ImpRatio'].tolist())
            idList = list(df.index.values)
            plt.scatter(idList, impRatioList, s=5, marker='.')
            plt.xlabel("Edge Index")
            plt.ylabel('$ER(I)_{it}^{AB}$')
            plt.rc('font', size=14)          # controls default text sizes
            plt.savefig("PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/Real" + str(file)[:-4] + ".png", bbox_inches='tight', format='png', dpi=400)
            plt.clf()
            

<Figure size 432x288 with 0 Axes>

In [12]:
# Not important - Just renaming the image files for latex

for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph'):
    if 'Log' in str(file):
        if 'INA' in str(file):
            code = str(file[20:-6])
            year = str(file[12:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + 'DIFINALog-' + code + '-' + year + '.png')
        elif 'ENA' in str(file):
            code = str(file[20:-6])
            year = str(file[12:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + 'DIFENALog-' + code + '-' + year + '.png')
    else:
        if 'INA' in str(file):
            code = str(file[21:-6])
            year = str(file[13:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + 'DIFINA-' + code + '-' + year + '.png')
        elif 'ENA' in str(file):
            code = str(file[21:-6])
            year = str(file[13:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry1Graph/' + 'DIFENA-' + code + '-' + year + '.png')


In [11]:
# Not important - Just renaming the image files for latex

for file in os.listdir('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph'):
    if 'Real' in str(file):
        if 'INA' in str(file):
            code = str(file[20:-6])
            year = str(file[12:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/' + 'ERINA-' + code + '-' + year + '.png')
        elif 'ENA' in str(file):
            code = str(file[20:-6])
            year = str(file[12:-14])
            os.rename('PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/' + file,'PycharmProjects/EnergyTradeAnalysis/Results/Assymetry2Graph/' + 'ERENA-' + code + '-' + year + '.png')
